## Setup

In [1]:
import os
import optuna
from dotenv import load_dotenv
from sklearn.metrics import f1_score, recall_score

from src.py_src import util
from src.py_src.models import GatekeeperModel

C:\Users\Eduardo\BES\IC\Solar_Flares\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load_dotenv()

slided_df_path = os.path.join(os.getenv("SLIDED_DFS_CSV_PATH"), "data_slided_V3.parquet")
target_class = 'target_class_in_24h'
target_flux = 'target_flux_in_24h'
target_columns = [target_class, target_flux]

df_model_input = util.create_df_model_input_opt(slided_df_path, target_columns, "xl_")

Carregando 34 colunas do arquivo Parquet...


## Preparing Data

In [7]:
gatekeeper_pool = df_model_input.copy()

train_pct = 0.7
val_pct = (1-train_pct)/2

data = util.prepare_data(
    df_model_input=gatekeeper_pool,
    target_class_col=target_class,
    lambda_function=lambda lb: 1 if lb >= 1 else 0,
    train_pct=train_pct,
    val_pct=val_pct,
    target_flux_col=target_flux
)

## Discovery Model

In [ ]:
discovery_model = GatekeeperModel(
    params={
        'n_estimators': 300,
        'learning_rate': 0.05,
        'max_depth': 5,
        'n_jobs': -1,
        'random_state': 42
    }
)

In [ ]:
selected_features = discovery_model.discover_top_features(
    x=data['x']['train'],
    y=data['y']['train'],
    flux_values=data['flux']['train'],
    cumulative_threshold=0.95
)

## Hyperparameter Tuning (Optuna)

In [ ]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'n_estimators': 1000,
        'random_state': 1502,
        'n_jobs': -1,
        'early_stopping_rounds': 50,
        'device': 'cuda',

        'scale_pos_weight': trial.suggest_float("scale_pos_weight", 1.0, 5.0),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'gamma': trial.suggest_float('gamma', 0.1, 5.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    model = GatekeeperModel(params=params, features_to_keep=selected_features)

    model.fit(
        x=data['x']['train'],
        y=data['y']['train'],
        eval_set=[(data['x']['val'], data['y']['val'])],
        verbose=False
    )

    y_pred_proba = model.predict_proba(data['x']['val'])[:, 1]
    y_pred_class = (y_pred_proba >= 0.5).astype(int)

    recall_flare = recall_score(data['y']['val'], y_pred_class, pos_label=1)
    recall_no_flare = recall_score(data['y']['val'], y_pred_class, pos_label=0)
    w = 2
    score = (w * recall_flare) + (1 * recall_no_flare)

    return score

In [ ]:
study = optuna.create_study(direction='maximize')
print("\nIniciando tuning...")
study.optimize(objective, n_trials=500)

print(f"\nBest Score: {study.best_value:.4f}")
best_params = study.best_params

best_params.update({
    'n_estimators': 1000, 'objective': 'binary:logistic',
    'eval_metric': 'logloss', 'random_state': 1502,
    'n_jobs': -1, 'early_stopping_rounds': 50
})

In [ ]:
final_model = GatekeeperModel(params=study.best_params, features_to_keep=selected_features)
final_model.fit(
    x=data['x']['train'], y=data['y']['train']
)

## Threshold Tuning

In [ ]:
fig = final_model.get_threshold_graph(data['x']['test'], data['y']['test'])
display(fig)

In [ ]:
final_model.optimize_threshold(data['x']['test'], data['y']['test'], target_recall=0.97)

## Results

In [ ]:
print(final_model.get_classification_report(
    data['x']['test'], data['y']['test'], target_names=['No Flare', 'Flare']
))

In [ ]:
fig, summary = final_model.analyze_flux_errors(
    data['x']['test'], data['y']['test'],
    flux_values=data['flux']['test']
)
display(summary)

In [ ]:
error_report = final_model.analyze_error_distribution(
    x=data['x']['test'],
    y_true=data['y']['test'],
    flux_values=data['flux']['test']
)
display(error_report)

## Features Importance

In [ ]:
features_importance = final_model.get_feature_importance()
features_importance

## Export

In [ ]:
save_path = r'../../models/gatekeeper/gatekeeper_model_v8.joblib'
final_model.save(save_path)